## Notebook to mosaic images captured within one minute of each other

Jukes Liu & Rainey Aberle

January 2022

In [ ]:
import os
import rasterio as rio
import numpy as np
import glob
import subprocess
from osgeo import gdal
import matplotlib.pyplot as plt

In [ ]:
# -----Path to images
site_name = 'Sperry'
base_path = '/Users/raineyaberle/Research/PhD/Planet_snow_cover/planet-snow/'
im_path = base_path+'../study-sites/'+site_name+'/imagery/PlanetScope/2016-2021/PSScene4Band/'

# -----Path for output images
out_path = im_path+'../../stitched/'

# -----Image file extension for mosaicing
ext = 'SR_clip'

### Mosaic image files

In [ ]:
# -----Create output directory if it does not already exist
if os.path.isdir(out_path)==0:
    os.mkdir(out_path)
    print(out_path+' directory made')

# -----Load file names with proper extension
os.chdir(im_path)
im_names = glob.glob('*'+ext+'*')
im_names.sort() 
    
# ----Grab all unique scenes (images captured within the same hour)
uniquescenes = [] 
for scene in im_names:
    date = scene[0:11]
    uniquescenes.append(date)
uniquescenes = list(set(uniquescenes))
uniquescenes.sort() # sort chronologically
uniquescenes

In [ ]:
# -----Loop through unique scenes
for scene in uniquescenes:
    filepaths = [] # files from the same hour to mosaic together
    for file in im_names: # check all files
        if (scene in file): # if they match the scene date 
            filepaths.append(im_path + file) # add the path to the file
    
    # construct the gdal_merge command
    cmd = 'gdal_merge.py -v '

    # add input files to command
    for filepath in filepaths:
        cmd += filepath+' '
    
    # define the out path with correct extension
    if ext == 'DN_udm.tif':
        out = os.path.join(out_path, scene + "_DN_mask.tif")
    elif ext == 'udm2.tif':
        out = os.path.join(out_path, scene + "_mask.tif")
    else:
        out = os.path.join(out_path, scene + ".tif")

    cmd += '-o '+out

    # run the command 
    p = subprocess.run(cmd, shell=True, capture_output=True) 
    print(p)

In [ ]:
# -----Print files in outpath to verify
os.chdir(out_path)
out_path_files = glob.glob('*.tif') # grab all files in out_path
out_path_files.sort() # sort by date
out_path_files # print

In [ ]:
# -----Plot image mosaics to preview
for im_name in out_path_files:
    im = rio.open(im_name)
    b = im.read(1)
    g = im.read(2)
    r = im.read(3)
    if np.nanmean(b)>1e3:
        im_scalar = 10000
        b = b / im_scalar
        g = g / im_scalar
        r = r / im_scalar
    plt.title(im_name[0:11])
    plt.imshow(np.dstack([r,g,b]))
    plt.show()

### Done!